In [ ]:
# Email Spam Classification with Large Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

print("Email Spam Classification with Large Dataset")
print("=" * 50)

# STEP 1: Load Dataset
def load_kaggle_dataset(file_path):
    """Load emails.csv dataset with enhanced error handling"""
    try:
        print(f"Attempting to load: {file_path}")
        
        # Load the CSV file
        df = pd.read_csv(file_path)
        print(f"✅ File loaded successfully! Shape: {df.shape}")
        print("📋 Columns found:", df.columns.tolist())
        
        # Show first few rows to understand structure
        print("\n📊 First 3 rows of your dataset:")
        print(df.head(3))
        
        # Common column name patterns for spam datasets
        column_mappings = [
            ('text', 'label'),
            ('message', 'label'), 
            ('email', 'spam'),
            ('Message', 'Category'),
            ('v2', 'v1'),  # SMS spam dataset format
            ('text', 'target'),
            ('content', 'class'),
            ('body', 'type'),
            ('email_text', 'is_spam'),
            ('content', 'label'),
            ('text', 'class')
        ]
        
        # Try to find the right columns automatically
        for text_col, label_col in column_mappings:
            if text_col in df.columns and label_col in df.columns:
                print(f"🎯 Found matching columns: '{text_col}' (text) and '{label_col}' (labels)")
                
                # Create standardized dataframe
                result_df = df[[text_col, label_col]].copy()
                result_df.columns = ['text', 'label']
                
                # Remove any missing values
                initial_size = len(result_df)
                result_df = result_df.dropna()
                if len(result_df) < initial_size:
                    print(f"⚠️ Removed {initial_size - len(result_df)} rows with missing values")
                
                # Ensure text column is string type
                result_df['text'] = result_df['text'].astype(str)
                
                # Show label distribution
                unique_labels = result_df['label'].unique()
                print(f"🏷️ Unique labels found: {unique_labels}")
                
                # Convert labels to 0/1 format
                if len(unique_labels) == 2:
                    if result_df['label'].dtype == 'object':
                        # Try different spam indicators
                        spam_indicators = ['spam', 'SPAM', 'Spam', '1', 1, 'positive', 'yes', 'Y']
                        ham_indicators = ['ham', 'HAM', 'Ham', '0', 0, 'negative', 'no', 'N']
                        
                        converted = False
                        for spam_word in spam_indicators:
                            if spam_word in unique_labels:
                                result_df['label'] = (result_df['label'] == spam_word).astype(int)
                                print(f"✅ Converted '{spam_word}' → 1 (spam), others → 0 (ham)")
                                converted = True
                                break
                        
                        if not converted:
                            # If no standard labels found, convert first unique value to 1
                            first_label = unique_labels[0]
                            result_df['label'] = (result_df['label'] == first_label).astype(int)
                            print(f"⚠️ Auto-converted '{first_label}' → 1, others → 0")
                            print("⚠️ Please verify this conversion is correct!")
                
                # Final validation
                spam_count = sum(result_df['label'])
                ham_count = len(result_df) - spam_count
                
                print(f"\n📈 Final dataset statistics:")
                print(f"   Total emails: {len(result_df)}")
                print(f"   Spam emails: {spam_count}")
                print(f"   Ham emails: {ham_count}")
                print(f"   Balance ratio: {spam_count/len(result_df):.2%} spam")
                
                if spam_count == 0 or ham_count == 0:
                    print("⚠️ WARNING: Only one class found! Check your labels.")
                
                return result_df
        
        # If no automatic match found, provide manual guidance
        print("\n❌ Could not automatically detect text and label columns.")
        print("\n📋 Your dataset columns:")
        for i, col in enumerate(df.columns):
            sample_values = df[col].dropna().head(3).tolist()
            print(f"   {i+1}. '{col}' - Sample: {sample_values}")
        
        print(f"\n🔧 Manual fix needed:")
        print(f"   1. Identify which column contains the email text")
        print(f"   2. Identify which column contains spam/ham labels")
        print(f"   3. Update the code:")
        print(f"      df = pd.read_csv('emails.csv')")
        print(f"      df = df[['your_text_column', 'your_label_column']]")
        print(f"      df.columns = ['text', 'label']")
        
        return None
        
    except FileNotFoundError:
        print(f"❌ File '{file_path}' not found!")
        print("📁 Make sure 'emails.csv' is in your current working directory.")
        
        # Show current directory and available files
        import os
        print(f"📂 Current directory: {os.getcwd()}")
        try:
            csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
            if csv_files:
                print(f"📄 CSV files found: {csv_files}")
            else:
                print("📄 No CSV files found in current directory")
        except:
            print("❌ Could not list directory contents")
        
        return None
        
    except pd.errors.EmptyDataError:
        print(f"❌ File '{file_path}' is empty!")
        return None
        
    except pd.errors.ParserError as e:
        print(f"❌ Error parsing CSV file: {e}")
        print("💡 Try opening the file in a text editor to check its format")
        return None
        
    except Exception as e:
        print(f"❌ Unexpected error loading dataset: {e}")
        print(f"💡 Error type: {type(e).__name__}")
        return None

# STEP 2: Create Sample Large Dataset (if you don't have one yet)
def create_sample_large_dataset():
    """Create a larger sample dataset for demonstration"""
    
    # Spam email templates
    spam_templates = [
        "FREE money! Click here to win ${}! Limited time offer!",
        "URGENT: Your account will be closed! Click link immediately!",
        "Congratulations! You've won {}! Send us your details!",
        "Buy cheap {} online! No prescription needed!",
        "SALE! {}% discount on everything! Buy now!",
        "Make ${} from home! Easy work!",
        "Lose {} pounds in 30 days! Guaranteed!",
        "HOT singles in your area! Click now!",
        "Your loan of ${} has been approved! Claim now!",
        "BREAKING: Celebrity secret revealed! Click here!"
    ]
    
    # Ham email templates
    ham_templates = [
        "Hi {}, let's meet for coffee tomorrow at {} PM",
        "Your order #{} has been shipped and will arrive by {}",
        "Team meeting scheduled for {} at {} AM in conference room",
        "Thanks for your presentation on {}, it was very informative",
        "Please review the attached {} document and provide feedback",
        "Your flight ticket confirmation for {} trip",
        "Happy {}! Hope you have a wonderful day",
        "The {} project deadline has been extended to next {}",
        "Reminder: Your appointment with {} is tomorrow at {}",
        "Invoice #{} for {} services is attached"
    ]
    
    emails = []
    labels = []
    
    # Generate 2500 emails of each type
    np.random.seed(42)
    
    # Generate spam emails
    for i in range(2500):
        template = np.random.choice(spam_templates)
        if '{}' in template:
            if 'money' in template or 'win' in template or 'loan' in template:
                email = template.format(np.random.randint(1000, 10000))
            elif 'discount' in template or 'pounds' in template:
                email = template.format(np.random.randint(10, 90))
            elif 'won' in template:
                prizes = ['iPhone', 'car', 'vacation', 'laptop', 'TV']
                email = template.format(np.random.choice(prizes))
            elif 'cheap' in template:
                products = ['medications', 'watches', 'electronics', 'supplements']
                email = template.format(np.random.choice(products))
            else:
                email = template
        else:
            email = template
        
        emails.append(email)
        labels.append(1)
    
    # Generate ham emails
    names = ['John', 'Sarah', 'Mike', 'Lisa', 'David', 'Emma', 'Chris', 'Anna']
    times = ['9', '10', '11', '2', '3', '4']
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    months = ['January', 'February', 'March', 'April', 'May', 'June']
    projects = ['marketing', 'development', 'research', 'design', 'sales']
    
    for i in range(2500):
        template = np.random.choice(ham_templates)
        if '{}' in template:
            if 'meet' in template:
                email = template.format(np.random.choice(names), np.random.choice(times))
            elif 'order' in template:
                email = template.format(np.random.randint(1000, 9999), np.random.choice(days))
            elif 'meeting' in template:
                email = template.format(np.random.choice(days), np.random.choice(times))
            elif 'presentation' in template:
                email = template.format(np.random.choice(projects))
            elif 'document' in template:
                docs = ['contract', 'report', 'proposal', 'budget', 'plan']
                email = template.format(np.random.choice(docs))
            elif 'flight' in template:
                email = template.format(np.random.choice(months))
            elif 'Happy' in template:
                events = ['birthday', 'anniversary', 'new year', 'holiday']
                email = template.format(np.random.choice(events))
            elif 'project' in template:
                email = template.format(np.random.choice(projects), np.random.choice(months))
            elif 'appointment' in template:
                email = template.format(np.random.choice(names), np.random.choice(times))
            elif 'Invoice' in template:
                services = ['consulting', 'design', 'development', 'marketing']
                email = template.format(np.random.randint(1000, 9999), np.random.choice(services))
            else:
                email = template
        else:
            email = template
            
        emails.append(email)
        labels.append(0)
    
    df = pd.DataFrame({'text': emails, 'label': labels})
    df['text'] = df['text'].astype(str)  # Ensure text column is string type
    return df

# Load or create dataset
print("Loading dataset...")
df = load_kaggle_dataset('emails.csv')

# If loading fails, create a sample dataset
if df is None:
    print("\nCreating sample dataset...")
    df = create_sample_large_dataset()
    print("✅ Sample dataset created with 5000 emails (2500 spam, 2500 ham)")

if df is not None:
    print(f"\nDataset loaded successfully!")
    print(f"Total emails: {len(df)}")
    print(f"Spam emails: {sum(df['label'])}")
    print(f"Ham emails: {len(df) - sum(df['label'])}")
    
    # STEP 3: Data Exploration
    print("\n" + "="*50)
    print("DATA EXPLORATION")
    print("="*50)
    
    # Basic statistics
    print(f"Dataset shape: {df.shape}")
    print(f"Missing values: {df.isnull().sum().sum()}")
    print(f"Class distribution:")
    print(df['label'].value_counts())
    
    # Text length analysis
    df['text_length'] = df['text'].str.len()
    print(f"\nAverage email length:")
    print(f"Spam: {df[df['label']==1]['text_length'].mean():.1f} characters")
    print(f"Ham: {df[df['label']==0]['text_length'].mean():.1f} characters")
    
    # STEP 4: Data Preprocessing and Feature Engineering
    print("\n" + "="*50)
    print("FEATURE ENGINEERING")
    print("="*50)
    
    # Add more features
    df['word_count'] = df['text'].str.split().str.len()
    df['exclamation_count'] = df['text'].str.count('!')
    df['question_count'] = df['text'].str.count('\?')
    df['uppercase_ratio'] = df['text'].str.count(r'[A-Z]') / df['text_length']
    df['dollar_count'] = df['text'].str.count('\$')
    
    # Prepare features
    X_text = df['text']
    X_numeric = df[['text_length', 'word_count', 'exclamation_count', 
                   'question_count', 'uppercase_ratio', 'dollar_count']]
    y = df['label']
    
    # Split the data
    X_text_train, X_text_test, X_num_train, X_num_test, y_train, y_test = train_test_split(
        X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\nTraining set size: {len(X_text_train)}")
    print(f"Test set size: {len(X_text_test)}")
    
    # Text vectorization with optimized parameters for large dataset
    print("\nConverting text to features...")
    vectorizer = TfidfVectorizer(
        stop_words='english',
        lowercase=True,
        max_features=5000,
        min_df=2,
        max_df=0.95,
        ngram_range=(1, 2)
    )
    
    X_text_train_tfidf = vectorizer.fit_transform(X_text_train)
    X_text_test_tfidf = vectorizer.transform(X_text_test)
    
    # Combine text features with numeric features
    from scipy.sparse import hstack
    X_train_combined = hstack([X_text_train_tfidf, X_num_train.values])
    X_test_combined = hstack([X_text_test_tfidf, X_num_test.values])
    
    print(f"\nCombined feature matrix shape: {X_train_combined.shape}")
    
    # STEP 5: Train Multiple Models
    print("\n" + "="*50)
    print("TRAINING MULTIPLE MODELS")
    print("="*50)
    
    models = {
        'Naive Bayes': MultinomialNB(),
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'SVM': SVC(probability=True, random_state=42)
    }
    
    model_scores = {}
    trained_models = {}
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        
        # Train model
        model.fit(X_train_combined, y_train)
        trained_models[name] = model
        
        # Cross-validation score
        cv_scores = cross_val_score(model, X_train_combined, y_train, cv=5)
        
        # Test set performance
        y_pred = model.predict(X_test_combined)
        test_accuracy = accuracy_score(y_test, y_pred)
        
        model_scores[name] = {
            'CV Mean': cv_scores.mean(),
            'CV Std': cv_scores.std(),
            'Test Accuracy': test_accuracy
        }
        
        print(f"  CV Score: {cv_scores.mean():.3f} (±{cv_scores.std():.3f})")
        print(f"  Test Accuracy: {test_accuracy:.3f}")
    
    # STEP 6: Detailed Evaluation of Best Model
    print("\n" + "="*50)
    print("DETAILED EVALUATION")
    print("="*50)
    
    # Find best model
    best_model_name = max(model_scores.keys(), 
                         key=lambda x: model_scores[x]['Test Accuracy'])
    best_model = trained_models[best_model_name]
    
    print(f"\nBest Model: {best_model_name}")
    print(f"  Cross-validation mean accuracy: {model_scores[best_model_name]['CV Mean']:.3f}")
    print(f"  Test accuracy: {model_scores[best_model_name]['Test Accuracy']:.3f}")
    
    # Detailed predictions
    y_pred_best = best_model.predict(X_test_combined)
    y_pred_proba = best_model.predict_proba(X_test_combined)
    
    # Classification report
    print(f"\nClassification Report for {best_model_name}:")
    print(classification_report(y_test, y_pred_best, target_names=['Ham', 'Spam']))
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_best)
    print(f"\nConfusion Matrix:")
    print(f"True Ham, Predicted Ham: {cm[0,0]}")
    print(f"True Ham, Predicted Spam: {cm[0,1]} (False Positives)")
    print(f"True Spam, Predicted Ham: {cm[1,0]} (False Negatives)")
    print(f"True Spam, Predicted Spam: {cm[1,1]}")
    
    # Visualize confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Ham', 'Spam'], 
                yticklabels=['Ham', 'Spam'])
    plt.title('Confusion Matrix')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()
    
    # STEP 7: Feature Importance Analysis
    if best_model_name == 'Logistic Regression':
        print("\n" + "="*50)
        print("TOP SPAM INDICATORS")
        print("="*50)
        
        feature_names = list(vectorizer.get_feature_names_out()) + \
                       ['text_length', 'word_count', 'exclamation_count', 
                        'question_count', 'uppercase_ratio', 'dollar_count']
        
        coefficients = best_model.coef_[0]
        
        # Top spam indicators
        top_spam_indices = coefficients.argsort()[-15:][::-1]
        print("\nTop 15 Spam Indicators:")
        for i, idx in enumerate(top_spam_indices, 1):
            print(f"{i:2d}. {feature_names[idx]:20s}: {coefficients[idx]:6.3f}")
        
        # Top ham indicators  
        top_ham_indices = coefficients.argsort()[:15]
        print("\nTop 15 Ham Indicators:")
        for i, idx in enumerate(top_ham_indices, 1):
            print(f"{i:2d}. {feature_names[idx]:20s}: {coefficients[idx]:6.3f}")
    
    # STEP 8: Test with New Emails
    print("\n" + "="*50)
    print("TESTING WITH NEW EMAILS")
    print("="*50)
    
    test_emails = [
        "FREE iPhone 15! Click now to claim your prize! Limited time!",
        "Hi John, can we reschedule our meeting to tomorrow at 3 PM?",
        "URGENT: Your bank account will be closed! Update now!",
        "Your Amazon order #12345 has been delivered to your address",
        "Make $5000 from home! Easy work, no experience needed!",
        "Team lunch tomorrow at the new restaurant downtown?",
        "CONGRATULATIONS! You've won $10000! Send your details now!",
        "Please find the quarterly report attached for your review"
    ]
    
    for email in test_emails:
        # Create features for the email
        email_df = pd.DataFrame({'text': [email]})
        email_df['text_length'] = email_df['text'].str.len()
        email_df['word_count'] = email_df['text'].str.split().str.len()
        email_df['exclamation_count'] = email_df['text'].str.count('!')
        email_df['question_count'] = email_df['text'].str.count('\?')
        email_df['uppercase_ratio'] = email_df['text'].str.count(r'[A-Z]') / email_df['text_length']
        email_df['dollar_count'] = email_df['text'].str.count('\$')
        
        # Transform text
        email_tfidf = vectorizer.transform(email_df['text'])
        email_numeric = email_df[['text_length', 'word_count', 'exclamation_count', 
                                 'question_count', 'uppercase_ratio', 'dollar_count']].values
        
        # Combine features
        email_combined = hstack([email_tfidf, email_numeric])
        
        # Predict
        prediction = best_model.predict(email_combined)[0]
        confidence = max(best_model.predict_proba(email_combined)[0])
        
        print(f"\nEmail: '{email[:60]}{'...' if len(email) > 60 else ''}'")
        print(f"Prediction: {'SPAM' if prediction == 1 else 'HAM'}")
        print(f"Confidence: {confidence:.2f}")
    
    print(f"\nModel training complete! Best model: {best_model_name}")
    print(f"Final accuracy: {model_scores[best_model_name]['Test Accuracy']:.3f}")
    
else:
    print("Failed to load dataset. Please check the file path and format.")


Email Spam Classification with Large Dataset
Loading dataset...
Attempting to load: emails.csv
✅ File loaded successfully! Shape: (5172, 3002)
📋 Columns found: ['Email No.', 'the', 'to', 'ect', 'and', 'for', 'of', 'a', 'you', 'hou', 'in', 'on', 'is', 'this', 'enron', 'i', 'be', 'that', 'will', 'have', 'with', 'your', 'at', 'we', 's', 'are', 'it', 'by', 'com', 'as', 'from', 'gas', 'or', 'not', 'me', 'deal', 'if', 'meter', 'hpl', 'please', 're', 'e', 'any', 'our', 'corp', 'can', 'd', 'all', 'has', 'was', 'know', 'need', 'an', 'forwarded', 'new', 't', 'may', 'up', 'j', 'mmbtu', 'should', 'do', 'am', 'get', 'out', 'see', 'no', 'there', 'price', 'daren', 'but', 'been', 'company', 'l', 'these', 'let', 'so', 'would', 'm', 'into', 'xls', 'farmer', 'attached', 'us', 'information', 'they', 'message', 'day', 'time', 'my', 'one', 'what', 'only', 'http', 'th', 'volume', 'mail', 'contract', 'which', 'month', 'more', 'robert', 'sitara', 'about', 'texas', 'nom', 'energy', 'pec', 'questions', 'www', 'd

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.